## Multirun experiment pipeline

The following notebook establishes a generalized pipeline for evaluating a computing reservoir against a given task, given multiple experimental runs of the same reservoir.


In [1]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src

### Importing the dataset

In [4]:
from src.model.rc_dataset import ExperimentDataset


csv_path = '../datasets/hydroshoot_large_trimmed.csv'
dataset = ExperimentDataset(csv_path=csv_path)
print(dataset)

Dataset properties:
	n_runs:      84
	n_steps:    168
	state_size: 360

Available targets: 
	input_Tac, input_u, input_hs, input_Rg, output_Rg, output_An, output_E, output_Tleaf

Available state variables: 
	state_An, state_E, state_Eabs, state_Ei, state_Flux, state_FluxC, state_Tlc, state_gb, state_gs, state_psi_head, state_u



### Selecting runs to use

In this notebook we use all runs.

In [8]:
RUN_IDS = np.arange(dataset.n_runs())

### Target definition

A target is defined as a transformation of a series.

In [ ]:
def direct_target(dataset: ExperimentDataset, target: str): pd.Series:
  """Returns a function that generates the target from the run id."""
  def get_target(run_id):
    return dataset.get_target(run_id, target)
  return get_target

### Data preprocessing

Preprocessing performed: 

In [ ]:
# ...

### Model definition

In [ ]:
# ...

### Train-test splitting

In [ ]:
# ...

### Grouping

In [ ]:
# ...

### Readout training

In [ ]:
# ...

### Store results